# OBJECTIVES


*Background:*
The company has collected a comprehensive set of sales data across various dimensions, including products, customers, regions, and time periods. The goal is to gain valuable insights into the sales performance and make data-driven decisions to optimize revenue and enhance overall business strategy.

*Objectives:*

1. **Sales Performance Overview:**
   - Understand the historical sales trends and patterns to inform future business strategies.

2. **Product Analysis:**
   - Identify top-performing products and product categories.
   - Explore opportunities for product portfolio enhancement and marketing strategies.

3. **Customer Segmentation:**
   - Segment customers based on their purchasing behavior and preferences.
   - Tailor marketing and engagement strategies for different customer segments.

4. **Geographical Analysis:**
   - Evaluate sales performance across different regions.
   - Identify regions with untapped potential or areas requiring special attention.

5. **Time-based Analysis:**
   - Analyze sales trends over different time dimensions (daily, monthly, yearly).
   - Identify peak sales periods and strategize inventory and marketing efforts accordingly.

6. **Promotion and Reseller Impact:**
   - Evaluate the effectiveness of promotions on sales.
   - Assess the contribution of different resellers to overall sales and optimize partnerships.

7. **Financial Insights:**
   - Calculate key financial metrics such as revenue, profit margins, and return on investment.
   - Identify opportunities for cost optimization and revenue growth.

*Outcome:*
By addressing the objectives outlined above, the company aims to enhance its understanding of the sales landscape, identify growth opportunities, and make informed decisions to improve overall business performance.

In [1]:
from pyspark.sql import SparkSession
import os

# Initialize a Spark session
spark = SparkSession.builder.appName("FileList").getOrCreate()

# Specify the folder path
folder_path = "C:/Users/neste/OneDrive/Desktop/karanja/DataSet_final/DataSet_final"

# List all files in the specified folder
file_list = [file for file in os.listdir(folder_path) if file.endswith(".csv")]

# Print the list of CSV files
print("List of CSV files in the folder:")
for file in file_list:
    print(file)


List of CSV files in the folder:
DimAccount.csv
DimCurrency.csv
DimCustomer.csv
DimDate.csv
DimDepartmentGroup.csv
DimGeography.csv
DimOrganization.csv
DimProduct.csv
DimProductCategory.csv
DimProductSubcategory.csv
DimPromotion.csv
DimReseller.csv
DimSalesReason.csv
DimSalesTerritory.csv
DimScenario.csv
FactCallCenter.csv
FactCurrencyRate.csv
FactFinance.csv
FactInternetSales.csv
FactSalesTargets.csv
